In [10]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json
import copy
import matplotlib
import time
import re
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [23]:
df = pd.DataFrame(columns=['question_index', 'question_polarity', 'context_condition', 'category', 
                      'stereotyped_group', 'context', 'question', 'ans0', 'ans1', 'ans2', 'label'])

In [27]:
def read_jsonl(fn):
    df = pd.DataFrame(columns=['question_index', 'question_polarity', 'context_condition', 'category', 
                      'stereotyped_group', 'context', 'question', 'ans0', 'ans1', 'ans2', 'label'])
    
    with open(f'./data/bbq/{fn}.jsonl', 'r') as json_file:
        json_list = list(json_file)

    for json_str in tqdm(json_list):
        result = json.loads(json_str)
        res = [result['question_index'], result['question_polarity'], result['context_condition'],
              result['category'], ','.join(result['additional_metadata']['stereotyped_groups']), result['context'], 
              result['question'], result['ans0'], result['ans1'], 
              result['ans2'], result['label']]

        df.loc[len(df)] = res

    return df

In [28]:
df = pd.concat([df, read_jsonl('Age')], ignore_index=True)
df = pd.concat([df, read_jsonl('Gender_identity')], ignore_index=True)
df = pd.concat([df, read_jsonl('Race_ethnicity')], ignore_index=True)

100%|█████████████████████████████| 6880/6880 [00:03<00:00, 1973.90it/s]


In [31]:
X, y = df[[i for i in df.columns if i != 'label' ]], df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

df_train['train'] = 1
df_test['train'] = 0
df = pd.concat([df_train, df_test], ignore_index=True)

In [41]:
rng = np.random.default_rng(12345)

In [42]:
df['hard_label'] = rng.random(len(df)) > 0.5

In [46]:
df.to_csv('data/processed/bbq.csv', index=False)